*<h3>Data Loading</h3>*

In [77]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

In [78]:
df = pd.read_csv('supermarket-sales.csv')
df.head()

,Order ID,Customer Name,Category,Sub Category,City,Order Date,Region,Sales,Discount,Profit,State
0,OD1,Harish,Oil & Masala,Masalas,Vellore,11-08-2017,North,1254,0.12,401.28,Tamil Nadu
1,OD2,Sudha,Beverages,Health Drinks,Krishnagiri,11-08-2017,South,749,0.18,149.80,Tamil Nadu
2,OD3,Hussain,Food Grains,Atta & Flour,Perambalur,06-12-2017,West,2360,0.21,165.20,Tamil Nadu
3,OD4,Jackson,Fruits & Veggies,Fresh Vegetables,Dharmapuri,10-11-2016,South,896,0.25,89.60,Tamil Nadu
4,OD5,Ridhesh,Food Grains,Organic Staples,Ooty,10-11-2016,South,2355,0.26,918.45,Tamil Nadu


*<h3>Preprocessing</h3>*
**1.Removing_useless_columns**

In [79]:
df = df.drop('State', axis=1)
df = df.drop('Customer Name', axis=1)
df = df.drop('Discount', axis=1)
df = df.drop('Order ID', axis=1)
df.rename(columns={'Sub Category':'Subcategory'},inplace=True)
df.head()

,Category,Subcategory,City,Order Date,Region,Sales,Profit
0,Oil & Masala,Masalas,Vellore,11-08-2017,North,1254,401.28
1,Beverages,Health Drinks,Krishnagiri,11-08-2017,South,749,149.80
2,Food Grains,Atta & Flour,Perambalur,06-12-2017,West,2360,165.20
3,Fruits & Veggies,Fresh Vegetables,Dharmapuri,10-11-2016,South,896,89.60
4,Food Grains,Organic Staples,Ooty,10-11-2016,South,2355,918.45


**2.Fetching_day_month_year_and_dropping_date**

In [80]:
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True, errors='coerce')
df['day'] = df['Order Date'].dt.day
df['month'] = df['Order Date'].dt.month
df['year'] = df['Order Date'].dt.year
df =df.drop('Order Date', axis=1)
df.head()

,Category,Subcategory,City,Region,Sales,Profit,day,month,year
0,Oil & Masala,Masalas,Vellore,North,1254,401.28,11.0,8.0,2017.0
1,Beverages,Health Drinks,Krishnagiri,South,749,149.80,11.0,8.0,2017.0
2,Food Grains,Atta & Flour,Perambalur,West,2360,165.20,6.0,12.0,2017.0
3,Fruits & Veggies,Fresh Vegetables,Dharmapuri,South,896,89.60,10.0,11.0,2016.0
4,Food Grains,Organic Staples,Ooty,South,2355,918.45,10.0,11.0,2016.0


**3.Unique_Values**

In [89]:
print(len(df.Category.unique()), 
      len(df.Subcategory.unique()), 
      len(df.Region.unique()), 
      len(df.year.unique()))

7 23 5 5
